# Защита персональных данных клиентов

# Оглавление

- [1. Введение](#1)
- [1.1. Формулировка хадачи](#1.1)
- [1.2. Описание данных](#1.2)
- [2. Решение](#2)
- [2.1. Аналитическое доказательство. Умножение матриц](#2.1)
- [2.2. Практическое доказательство](#2.2)
- [2.2.1. Загрузка данных](#2.2.1)
- [2.2.2. Прогнозирование на исходной матрице признаков](#2.2.2)
- [2.2.3. Прогнозирование на матрице признаков, умноженной на обратимую матрицу](#2.2.3)
- [2.2.4. Прогнозирование на матрице признаков, умноженной на НЕобратимую матрицу](#2.2.4)
- [3. Выводы](#3)

# 1 Введение
<a id="1"></a>

# 1.1. Формулировка задачи
<a id="1.1"></a>

Необходимо защитить данные клиентов страховой компании «Хоть потоп» и разработайть такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1.2. Описание данных
<a id="1.2"></a>

Набор данных находится в файле /datasets/insurance.csv. Скачать датасет.
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

# 2. Решение
<a id="2"></a>

Для выполнения задачи предложим следующее решение: умножим исходную матрицу признаков на обратимую матрицу. Для функции потерь принятой равной $w = (X^T X)^{-1} X^T y$ значение результата должно сохраниться прежним.

## 2.1. Аналитическое доказательство. Умножение матриц <a id="2.1"></a>

**Обозначения:**

- $X$ — матрица признаков (нулевой столбец состоит из единиц)
- $y$ — вектор целевого признака
- $P$ — матрица, на которую умножаются признаки
- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу). Вектор весов находится по формуле $w = (X^T X)^{-1} X^T y$
- $a$ - ветокр предсказаний. находится по формуле $a = Xw$

задача обучения - найти $w$ при котором среднеквадратичная ошибка будет минимальной.  $w = \arg\min_w MSE(Xw, y)$

**Свойства матриц:**
$$
(AB)^T=B^TA^T   \\
(AB)^{-1}=B^{-1}A^{-1} \\
AA^{-1}=A^{-1}A=E \\
AE=EA=A \\
$$


**Сформулируем гепотезу:** 

При умножении матрицы признаков на обратимую матрицу вектор предсказний для модели машинного обучения на базе линейной регрессии останется прежним. 

**Сформулируем задачу:**  
**Дано:** 
1. Вектор предсказаний без умножения матрицы признаков $a_1 = Xw_1 + w_0$, где $X$ - матрица признаков
2. Вектор предсказаний с умноженем матрицы признаков $a_2 = XPw_2 + w_0$, где $X$ - матрица признаков, $P$ - обратимая матрица (определитель д.б. > 0)
3. Вектор весов линейной регрессии для матрицы без умножения матрицы признаков $w_1 = (X^T X)^{-1} X^T y$
4. Вектор весов линейной регрессии для матрицы с умножением матрицы признаков $w_2 = ((XP)^T XP)^{-1} (XP)^T y$

**Доказать:** $a_1 = a_2$

**Решение**

$$
1. Преобразуем \: вектор \: весов \: w_2:\\
w_2 = ((XP)^T XP)^{-1} (XP)^T y =>\\
      (P^T X^T  XP)^{-1} P^T X^T y =>\\
      (X^T X)^{-1} (P^T P)^{-1} P^T X^T y =>\\
      (X^T X)^{-1} P^{-1} (P^T)^{-1}  P^T X^T  y =>\\
      (X^T X)^{-1} P^{-1} E X^T  y =>\\
      (X^T X)^{-1} P^{-1} X^T y\\
\\
2. Выразим \: a_2 \: через \: w_2:\\
a_2 = XPw_2 + w_0 =>\\
a_2 = XP (X^T X)^{-1} P^{-1} X^T y + w_0 =>\\
a_2 = X  (X^T X)^{-1} E X^T y + w_0 =>\\
a_2 = X  (X^T X)^{-1} X^T y + w_0 =>\\
\\
3. Выразим \: a_1 \: через \: w_1:\\
a_1 = Xw_1 + w_0 =>\\
a_1 = X (X^T X)^{-1} X^T y + w_0
\\
4. Таким \: образом\\ 
a_1 = a_2\\ 
Ч.Т.Д.
$$

## 2.2. Практическое доказательство
<a id="2.2"></a>

## 2.2.1. Загрузка данных
<a id="2.2.1"></a>

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [2]:
path1 = 'insurance.csv'
path2 = '/datasets/insurance.csv'
if os.path.exists(path1):
    df = pd.read_csv(path1)
elif os.path.exists(path2):
    df = pd.read_csv(path2)
else: print('файл не найден')

In [3]:
display(df.head())
print(df.info())
print('Список возможных значений в целевом признаке:', df['Страховые выплаты'].unique())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
Список возможных значений в целевом признаке: [0 1 2 3 5 4]


## 2.2.2. Прогнозирование на исходной матрице признаков
<a id="2.2.2"></a>

In [4]:
features = df.drop(columns=['Страховые выплаты'])
target = df['Страховые выплаты']
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
#features_scaled = features # эту строчку удалить потом

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, random_state = 2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3750, 4)
(1250, 4)
(3750,)
(1250,)


In [14]:
model = LinearRegression()
model.fit(X_train, y_train)
prediction_1 = model.predict(X_test)
r2_score_1 = r2_score(y_test, prediction_1)
print('r2 =',r2_score_1)

r2 = 0.4049827646604208


## 2.2.3. Прогнозирование на матрице признаков, умноженной на обратимую матрицу 
<a id="2.2.3"></a>

In [6]:
# зафиксируем random_state 
np.random.seed(1)
# создадим обратимую матрицу размерностью 4x4
P = np.random.rand(4,4)

print('Проверим, равен ли определитель матрицы нулю:', np.linalg.det(P)==0)

# умножим ранее отмасштмабированные признаки на обратимую матрицу
features_scaled_mutiplied = features_scaled.dot(P)
target = df['Страховые выплаты']

X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(features_scaled_mutiplied, target, random_state = 2)

print(X_train_multi.shape)
print(X_test_multi.shape)
print(y_train_multi.shape)
print(y_test_multi.shape)

Проверим, равен ли определитель матрицы нулю: False
(3750, 4)
(1250, 4)
(3750,)
(1250,)


In [13]:
model.fit(X_train_multi, y_train_multi)
prediction_2 = model.predict(X_test_multi)

r2_score_2 = r2_score(y_test_multi, prediction_2)
print('r2 =',r2_score_2)

r2 = 0.40498276466042094


**Вывод** результат при умножении на обратимую матрицу не отличается от результа без умножения

## 2.2.4. Прогнозирование на матрице признаков, умноженной на НЕобратимую матрицу 
<a id="2.2.4"></a>

In [10]:
# создадим необратимую матрицу
sing_matrix = np.array([[0,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]])
#B_inv = np.linalg.inv(B) 

print('Проверим, равен ли определитель матрицы нулю:', np.linalg.det(sing_matrix)==0)

# умножим ранее отмасштмабированные признаки на НЕобратимую матрицу
features_scaled_mutiplied_sing = features_scaled.dot(sing_matrix)
target = df['Страховые выплаты']

X_train_multi_sing, X_test_multi_sing, y_train_multi_sing, y_test_multi_sing = train_test_split(features_scaled_mutiplied_sing, target, random_state = 2)

print(X_train_multi_sing.shape)
print(X_test_multi_sing.shape)
print(y_train_multi_sing.shape)
print(y_test_multi_sing.shape)

Проверим, равен ли определитель матрицы нулю: True
(3750, 4)
(1250, 4)
(3750,)
(1250,)


In [12]:
model.fit(X_train_multi_sing, y_train_multi_sing)
prediction_3 = model.predict(X_test_multi_sing)

r2_score_3 = r2_score(y_test_multi_sing, prediction_3)
print('r2 =',r2_score_3)

r2 = 0.12984892573149442


## 3. Выводы
<a id="3"></a>

В рамаках данного проекта было доказано аналитически и эмпирически, что при прогнозировании при помощи линейной регрессии при умножении исходных признаков на обратимую матрицу вектор предсказаний остается неизменным. И наоборот, при умножении на сингулярную матрицу (не обратимую) вектор предсказаний отличается. В случае домножения матрицы признаков на случайную обратимую матрицу можно обеспечить защиту данных.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [ ]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования